In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/PBL7/Tram

Mounted at /content/drive
/content/drive/MyDrive/PBL7/Tram


In [ ]:
!pip install cloudscraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.2 MB/s eta 0:00:00


# 1. Crawl book-list

In [ ]:
import time
import cloudscraper
from collections import deque
from bs4 import BeautifulSoup

country_dict = {'New Zealand': 'https://www.bookcrossing.com/hunt/11/',
                  'Australia': 'https://www.bookcrossing.com/hunt/4/',
                  'England': 'https://www.bookcrossing.com/hunt/3/',
                  'Ireland': 'https://www.bookcrossing.com/hunt/13/',
                  'USA': 'https://www.bookcrossing.com/hunt/1/',
                  'Canada': 'https://www.bookcrossing.com/hunt/2/'}

hrefs_queue = deque()
hrefs_queue.extend(country_dict.values())
max_attempts = 10
notifi_count = 0

book_list_link = []

# Lặp qua số lần thử
while len(hrefs_queue) != 0:
    url = hrefs_queue.pop()
    for attempt in range(max_attempts):
        # Tạo một cloudscraper instance
        scraper = cloudscraper.create_scraper()

        # Lấy nội dung trang web
        response = scraper.get(url)

        # Kiểm tra nếu yêu cầu thành công
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # If class book-list exist
            if soup.find('div', class_="book-list"):
                country = ""
                num_pages = -1

                # Find book COUNTRY
                p_country = soup.find('p', id='pBookSortBar')
                a_country = p_country.find_all('a')
                if len(a_country) > 1:
                    a_country_text = a_country[1].text
                    country = a_country_text
                else:
                    print("Không tìm thấy country.")

                # Find NUM_PAGES
                page_items = soup.find_all('li', class_='page-item')
                if len(page_items) > 1:
                    # Lấy chỉ số của page-item áp chót
                    # Bỏ qua thẻ cuối (là "»")
                    second_last_item = page_items[-2]

                    try:
                        second_last_page_number = int(second_last_item.find('a').text)
                        num_pages = second_last_page_number
                    except ValueError:
                        print("Không tìm thấy số trang hợp lệ trong thẻ áp chót.")
                else:
                    num_pages = 1
                book_list_link.append((url, a_country_text, num_pages))
            else:
                location_tables = soup.find_all('table', class_=lambda x: x and 'table table-sm table-striped table-part' in x)
                all_a_elements = []
                for location_table in location_tables:
                    all_a_elements.extend(location_table.find_all('a'))
                url_base = f'https://www.bookcrossing.com'
                current_hrefs = list(map(lambda x: url_base + x.get('href'), all_a_elements))
                hrefs_queue.extend(current_hrefs)
            notifi_count += 1
            if notifi_count == 100:
                print(f"Current queue size: {len(hrefs_queue)}")
                print(f"Current book list size: {len(book_list_link)}")
                notifi_count = 0
            break
        else:
            print(f"Không thể truy cập trang web, mã trạng thái: {response.status_code}")

        # Chờ 5 giây trước khi thử lại
        time.sleep(5)

Current queue size: 10
Current book list size: 63
Current queue size: 52
Current book list size: 124
Current queue size: 44
Current book list size: 199
Current queue size: 41
Current book list size: 252
Current queue size: 31
Current book list size: 315
Current queue size: 26
Current book list size: 383
Current queue size: 28
Current book list size: 443
Current queue size: 23
Current book list size: 509
Current queue size: 22
Current book list size: 569
Current queue size: 25
Current book list size: 648
Current queue size: 25
Current book list size: 738
Current queue size: 48
Current book list size: 796
Current queue size: 58
Current book list size: 849
Current queue size: 36
Current book list size: 907
Current queue size: 36
Current book list size: 958
Current queue size: 13
Current book list size: 1011
Current queue size: 20
Current book list size: 1065
Current queue size: 19
Current book list size: 1121


In [ ]:
len(book_list_link)

1171

In [ ]:
import pandas as pd

df = pd.DataFrame(book_list_link, columns=['url', 'country', 'num_pages'])
df.to_csv('book_list_link.csv', index=False)

# 2. Crawl Book-Link

In [ ]:
book_list_link = pd.read_csv('book_list_link.csv', index_col=False)
print(book_list_link.shape)
book_list_link.head()

(1171, 4)


,id,url,country,num_pages
0,0,https://www.bookcrossing.com/hunt/2/64/97938/8...,Canada,2
1,1,https://www.bookcrossing.com/hunt/2/64/156790/...,Canada,7
2,2,https://www.bookcrossing.com/hunt/2/63/35515/6...,Canada,2
3,3,https://www.bookcrossing.com/hunt/2/63/35515/8...,Canada,1
4,4,https://www.bookcrossing.com/hunt/2/63/30983/8...,Canada,3


In [ ]:
# Tạo file CSV và ghi header
header_df = pd.DataFrame(columns=['book_url', 'book_list_id'])
header_df.to_csv('book_link.csv', mode='w', header=True, index=False)

In [ ]:
import time
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd

book_link = []
url_base = 'https://www.bookcrossing.com'
count_book_link = 0
max_attempts = 10

def save_links_to_csv(book_link):
    book_link_df = pd.DataFrame(book_link, columns=['book_url', 'book_list_id'])
    book_link_df.to_csv('book_link.csv', mode='a', header=False, index=False)



for id, url, _, num_pages in book_list_link.itertuples(index=False):
    for page in range(1, num_pages + 1):
        for attempt in range(max_attempts):
            current_url = f"{url}?page={page}"
            # Tạo một cloudscraper instance
            scraper = cloudscraper.create_scraper()

            # Lấy nội dung trang web
            response = scraper.get(current_url)

            # Kiểm tra nếu yêu cầu thành công
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')

                # If class book-list exists
                if soup.find('div', class_="book-list"):
                    book_list_rows = soup.find_all('div', class_='book-list-row')
                    for row in book_list_rows:
                        all_links = row.select('a')
                        book_link_tag = all_links[-1]
                        if book_link_tag:
                            book_link.append((url_base + book_link_tag['href'], id))

                if len(book_link) >= 1000:
                    save_links_to_csv(book_link)
                    count_book_link += len(book_link)
                    print(f"Saved {len(book_link)} books to CSV. Current count: {count_book_link}")
                    book_link = []

                break
            else:
                print(f"Không thể truy cập trang web, mã trạng thái: {response.status_code}")
                if attempt == max_attempts - 1:
                    print(f"Nên thử lại sau! url: {current_url} sẽ được bỏ qua.")

            # Chờ 3 giây trước khi thử lại
            time.sleep(3)

# Save any remaining links that did not reach the 1000 count
if book_link:
    save_links_to_csv(book_link)
    print(f"Final book link size: {len(book_link)} (saved to CSV)")

Saved 1018 books to CSV. Current count: 1018
Saved 1023 books to CSV. Current count: 2041
Saved 1022 books to CSV. Current count: 3063
Saved 1022 books to CSV. Current count: 4085
Saved 1010 books to CSV. Current count: 5095
Saved 1002 books to CSV. Current count: 6097
Saved 1019 books to CSV. Current count: 7116
Saved 1010 books to CSV. Current count: 8126
Saved 1011 books to CSV. Current count: 9137
Saved 1003 books to CSV. Current count: 10140
Saved 1000 books to CSV. Current count: 11140
Saved 1010 books to CSV. Current count: 12150
Saved 1019 books to CSV. Current count: 13169
Saved 1000 books to CSV. Current count: 14169
Saved 1000 books to CSV. Current count: 15169
Saved 1023 books to CSV. Current count: 16192
Saved 1003 books to CSV. Current count: 17195
Saved 1021 books to CSV. Current count: 18216
Saved 1021 books to CSV. Current count: 19237
Saved 1011 books to CSV. Current count: 20248
Saved 1023 books to CSV. Current count: 21271
Saved 1020 books to CSV. Current count: 222

In [ ]:
import pandas as pd
book_link_df = pd.DataFrame(book_link, columns=['book_url', 'book_list_id'])
print(book_link_df.shape)
book_link_df.head()

In [ ]:
book_link_df.to_csv('book_link.csv', index=False)

# 3. Crawl book-detail



In [ ]:
book_link = pd.read_csv('book_link.csv', index_col=False)
print(book_link.shape)
book_link.head()

(203497, 2)


,book_url,book_list_id
0,https://www.bookcrossing.com/title/6330328,0
1,https://www.bookcrossing.com/title/1712457,0
2,https://www.bookcrossing.com/title/1221204,0
3,https://www.bookcrossing.com/title/6331305,0
4,https://www.bookcrossing.com/title/6330373,0


In [ ]:
book_link = book_link.drop_duplicates(subset='book_url', keep='first')
print(book_link.shape)
book_link.head()

(171286, 2)


,book_url,book_list_id
0,https://www.bookcrossing.com/title/6330328,0
1,https://www.bookcrossing.com/title/1712457,0
2,https://www.bookcrossing.com/title/1221204,0
3,https://www.bookcrossing.com/title/6331305,0
4,https://www.bookcrossing.com/title/6330373,0


In [ ]:
num_book_crawled = 132900
current_index = 164868
book_link = book_link.iloc[current_index:]

In [ ]:
# Tạo file CSV và ghi header
# header_df = pd.DataFrame(columns=['isbn', 'url', 'title', 'image_url', 'author', 'category', 'avg_rating', 'book_list_id'])
# header_df.to_csv('book_detail.csv', mode='w', header=True, index=False)

In [ ]:
import time
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd

book_detail = []
url_base = 'https://www.bookcrossing.com'
max_attempts = 10

def save_to_csv(book_detail):
    book_detail_df = pd.DataFrame(book_detail, columns=['isbn', 'url', 'title', 'image_url', 'author', 'category', 'avg_rating', 'book_list_id'])
    book_detail_df.to_csv('book_detail.csv', mode='a', header=False, index=False)

def error_save_to_csv(book_url, book_list_id):
    # append records to error file
    error_df = pd.DataFrame([[book_url, book_list_id]], columns=['book_url', 'book_list_id'])
    error_df.to_csv('error_url.csv', mode='a', header=False, index=False)


for url, book_list_id in book_link.itertuples(index=False):
    current_index += 1
    for attempt in range(max_attempts):
        # Tạo một cloudscraper instance
        scraper = cloudscraper.create_scraper()

        # Lấy nội dung trang web
        response = scraper.get(url)
        scraper.close()

        # Kiểm tra nếu yêu cầu thành công
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            isbn = ''
            title = ''
            image_url = ''
            author = ''
            category = ''
            avg_rating = ''
             # Extracting the ISBN
            find_isbn = soup.find(text=lambda text: text and 'ISBN:' in text)
            if find_isbn:
                isbn = find_isbn.split('ISBN: ')[1]
                isbn = isbn.strip()

            # Extracting the title
            find_title = soup.find('h2')
            if find_title:
                title = find_title.text.strip()

            # Extracting the image URL
            find_image = soup.find('img', {'class': 'book-journal'})
            if find_image:
                image_url = find_image['src']

            # Extracting the author
            find_author = soup.find('a', href=lambda href: href and 'author' in href)
            if find_author:
                author = find_author.text.strip()

            # Extracting the category
            find_category = soup.find('a', href=lambda href: href and 'category' in href)
            if find_category:
                category = find_category.text.strip()

            # Calculating the average rating
            find_rating = soup.select_one('.bookrating')
            if find_rating:
                rating_title = find_rating['title']
                avg_rating = int(rating_title.split()[1])

            if isbn:
                book_detail.append((isbn, url, title, image_url, author, category, avg_rating, book_list_id))
                if len(book_detail) == 100:
                    save_to_csv(book_detail)
                    num_book_crawled += 100
                    print(f"Saved 100 books to CSV. Current count: {num_book_crawled}. Current index: {current_index}.")
                    book_detail = []
            break
        else:
            print(f"Không thể truy cập trang web, mã trạng thái: {response.status_code}")
            if attempt == max_attempts - 1:
                print(f"Nên thử lại sau! Url: {url} sẽ được bỏ qua")
                error_save_to_csv(url, book_list_id)


        # Chờ 3 giây trước khi thử lại
        time.sleep(3)

# Save any remaining books that did not reach the 100 count
if book_detail:
    save_to_csv(book_detail)
    print(f"Final book detail size: {len(book_detail)} (saved to CSV)")


<ipython-input-10-442292208d21>:40: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  find_isbn = soup.find(text=lambda text: text and 'ISBN:' in text)


Saved 100 books to CSV. Current count: 133000. Current index: 164969.
Saved 100 books to CSV. Current count: 133100. Current index: 165074.
Saved 100 books to CSV. Current count: 133200. Current index: 165179.
Saved 100 books to CSV. Current count: 133300. Current index: 165283.
Saved 100 books to CSV. Current count: 133400. Current index: 165396.
Saved 100 books to CSV. Current count: 133500. Current index: 165500.
Saved 100 books to CSV. Current count: 133600. Current index: 165604.
Saved 100 books to CSV. Current count: 133700. Current index: 165708.
Saved 100 books to CSV. Current count: 133800. Current index: 165811.
Saved 100 books to CSV. Current count: 133900. Current index: 165914.
Saved 100 books to CSV. Current count: 134000. Current index: 166015.
Saved 100 books to CSV. Current count: 134100. Current index: 166115.
Saved 100 books to CSV. Current count: 134200. Current index: 166220.
Saved 100 books to CSV. Current count: 134300. Current index: 166332.
Saved 100 books to C

In [14]:
book_detail = pd.read_csv('book_detail.csv', index_col=False)
print(book_detail.shape)
book_detail.head()

(138180, 8)


,isbn,url,title,image_url,author,category,avg_rating,book_list_id
0,0889610487,https://www.bookcrossing.com/title/6330328,One Proud Summer,https://books.google.com/books/content?id=NxTh...,Marsha Hewitt,Children's Books,8.0,0
1,0394280741,https://www.bookcrossing.com/title/1712457,The Hollow Tree,https://books.google.com/books/content?id=Vaov...,Janet Lunn,Children's Books,8.0,0
2,0773770208,https://www.bookcrossing.com/title/1221204,One Man's Island,https://images-na.ssl-images-amazon.com/images...,David Conover,Literature & Fiction,NaN,0
3,0888330162,https://www.bookcrossing.com/title/6330373,Outdoors West,https://images-na.ssl-images-amazon.com/images...,Ian Bickle,Outdoors & Nature,8.0,0
4,155054442X,https://www.bookcrossing.com/title/6134748,The Secret Lives of Sgt. John Wilson,https://books.google.com/books/content?id=uQzD...,Lois Simmie,History,7.0,0
